In [ ]:
import pandas as pd
import sys
import os
from pathlib import Path

# Agregar el directorio raíz al path para encontrar src
# Esto asume que el notebook está en la carpeta 'notebooks/'
current_dir = Path.cwd()
project_root = current_dir.parent
sys.path.append(str(project_root))

# --- AQUÍ ESTÁ EL CAMBIO ---
# En lugar de 'from src.utils.db import get_db'
from src.utils.db import SupplyChainDB 

# Configuración visual
pd.set_option('display.max_columns', None)

# Inicializar la conexión
db = SupplyChainDB()
print("✅ Conexión establecida")

In [ ]:
import sys
import os

# 1. Agregar el directorio padre al path de Python (para encontrar src)
sys.path.append('..')

# 2. CAMBIO CRÍTICO: Mover el contexto de ejecución a la raíz del proyecto
# Esto hace que las rutas relativas 'data/processed/...' funcionen correctamente
if os.getcwd().endswith('notebooks'):
    os.chdir('..')

print(f"📂 Directorio de trabajo actual: {os.getcwd()}")

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json

from src.utils.db import SupplyChainDB

# Setup
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load database (Ahora sí encontrará el archivo lleno)
db = SupplyChainDB()

# Load baseline metrics
with open('data/processed/baseline_metrics.json', 'r') as f: # Nota: quité el '../' porque ya estamos en root
    baseline = json.load(f)

In [ ]:
import sys
import os

# 1. Agregar el directorio padre al path de Python (para encontrar src)
sys.path.append('..')

# 2. CAMBIO CRÍTICO: Mover el contexto de ejecución a la raíz del proyecto
# Esto hace que las rutas relativas 'data/processed/...' funcionen correctamente
if os.getcwd().endswith('notebooks'):
    os.chdir('..')

print(f"📂 Directorio de trabajo actual: {os.getcwd()}")

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import json

from src.utils.db import SupplyChainDB

# Setup
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Load database (Ahora sí encontrará el archivo lleno)
db = SupplyChainDB()

# Load baseline metrics
with open('data/processed/baseline_metrics.json', 'r') as f: # Nota: quité el '../' porque ya estamos en root
    baseline = json.load(f)

print("="*70)
print("TECHGEAR SUPPLY CHAIN - DATA EXPLORATION")
print("="*70)
print(f"\nBaseline Metrics:")
for key, value in baseline.items():
    if 'pct' in key:
        print(f"  {key:30s}: {value:.2f}%")
    elif isinstance(value, float):
        print(f"  {key:30s}: ${value:,.2f}")
    else:
        print(f"  {key:30s}: {value:,}")
        
        
# %% [markdown]
# # 1. Revenue Trend Analysis
# 
# Validating 15% YoY growth and identifying patterns

# %% Revenue over time
revenue_daily = db.query("""
    SELECT 
        date,
        SUM(revenue) as daily_revenue,
        SUM(units_sold) as daily_units,
        COUNT(*) as num_transactions
    FROM sales
    GROUP BY date
    ORDER BY date
""")

# Calculate 7-day moving average
revenue_daily['revenue_7d_ma'] = revenue_daily['daily_revenue'].rolling(7).mean()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=revenue_daily['date'],
    y=revenue_daily['daily_revenue'],
    mode='lines',
    name='Daily Revenue',
    line=dict(color='lightblue', width=1),
    opacity=0.5
))

fig.add_trace(go.Scatter(
    x=revenue_daily['date'],
    y=revenue_daily['revenue_7d_ma'],
    mode='lines',
    name='7-Day Moving Average',
    line=dict(color='blue', width=2)
))

fig.update_layout(
    title='Daily Revenue Trend (2023-2024)',
    xaxis_title='Date',
    yaxis_title='Revenue ($)',
    hovermode='x unified',
    height=500
)

fig.show()

# Calculate growth
revenue_daily['year'] = pd.to_datetime(revenue_daily['date']).dt.year
yearly = revenue_daily.groupby('year')['daily_revenue'].sum()
growth_rate = (yearly[2024] / yearly[2023] - 1) * 100

print(f"\n📈 Year-over-Year Growth: {growth_rate:.2f}%")
print(f"   2023 Total Revenue: ${yearly[2023]:,.2f}")
print(f"   2024 Total Revenue: ${yearly[2024]:,.2f}")


# %% [markdown]
# # 2. Seasonality Analysis
# 
# Identifying peak seasons and demand patterns

# %% Monthly seasonality
revenue_daily['month'] = pd.to_datetime(revenue_daily['date']).dt.month
revenue_daily['year'] = pd.to_datetime(revenue_daily['date']).dt.year

monthly_by_year = revenue_daily.groupby(['year', 'month'])['daily_revenue'].mean().reset_index()

fig = px.line(
    monthly_by_year,
    x='month',
    y='daily_revenue',
    color='year',
    title='Average Daily Revenue by Month',
    labels={'daily_revenue': 'Avg Daily Revenue ($)', 'month': 'Month'},
    markers=True
)

fig.update_xaxes(
    tickmode='array',
    tickvals=list(range(1, 13)),
    ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
)

fig.show()

# Peak months
monthly_avg = revenue_daily.groupby('month')['daily_revenue'].mean()
peak_month = monthly_avg.idxmax()
lowest_month = monthly_avg.idxmin()
peak_ratio = monthly_avg.max() / monthly_avg.min()

print(f"\n📊 Seasonality Metrics:")
print(f"   Peak month: {peak_month} (${monthly_avg.max():,.2f}/day)")
print(f"   Lowest month: {lowest_month} (${monthly_avg.min():,.2f}/day)")
print(f"   Peak/Low ratio: {peak_ratio:.2f}x")


# %% [markdown]
# # 3. External Events Impact
# 
# Quantifying the effect of Black Friday, supply disruptions, etc.

# %% Events impact analysis
events = db.query("SELECT * FROM external_events")
sales_with_events = db.query("""
    SELECT 
        s.date,
        SUM(s.revenue) as daily_revenue,
        e.type as event_type,
        e.description
    FROM sales s
    LEFT JOIN external_events e 
        ON s.date >= CAST(e.date AS DATE)
        AND s.date <= CAST(e.date AS DATE) + INTERVAL (e.duration_days) DAY
    GROUP BY s.date, e.type, e.description
    ORDER BY s.date
""")

# Focus on major events
major_events = {
    'Black-Friday': ('2023-11-24', 4),
    'Back-to-School': ('2023-08-01', 45),
    'Supply-Disruption': ('2024-03-10', 21)
}

for event_name, (start_date, duration) in major_events.items():
    start = pd.to_datetime(start_date)
    end = start + pd.Timedelta(days=duration)
    
    event_revenue = revenue_daily[
        (revenue_daily['date'] >= start) & 
        (revenue_daily['date'] <= end)
    ]['daily_revenue'].mean()
    
    # Baseline: 2 weeks before event
    baseline_start = start - pd.Timedelta(days=14)
    baseline_end = start - pd.Timedelta(days=1)
    baseline_revenue = revenue_daily[
        (revenue_daily['date'] >= baseline_start) & 
        (revenue_daily['date'] <= baseline_end)
    ]['daily_revenue'].mean()
    
    impact = (event_revenue / baseline_revenue - 1) * 100
    
    print(f"\n{event_name}:")
    print(f"   Baseline (2 weeks prior): ${baseline_revenue:,.2f}/day")
    print(f"   During event: ${event_revenue:,.2f}/day")
    print(f"   Impact: {impact:+.1f}%")
    
    
    
# %% [markdown]
# # 4. Product Performance Analysis
# 
# Which products drive revenue and which have issues?

# %% Top products by revenue
top_products = db.query("""
    SELECT 
        p.product_id,
        p.name,
        p.category,
        p.tier,
        SUM(s.revenue) as total_revenue,
        SUM(s.units_sold) as total_units,
        SUM(s.profit) as total_profit,
        COUNT(DISTINCT s.date) as days_with_sales,
        AVG(s.unit_price) as avg_price
    FROM sales s
    JOIN products p ON s.product_id = p.product_id
    GROUP BY p.product_id, p.name, p.category, p.tier
    ORDER BY total_revenue DESC
    LIMIT 20
""")

fig = px.bar(
    top_products,
    x='name',
    y='total_revenue',
    color='category',
    title='Top 20 Products by Revenue',
    labels={'total_revenue': 'Total Revenue ($)', 'name': 'Product'},
    text='total_revenue'
)

fig.update_traces(texttemplate='$%{text:,.0f}', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

print("\n🏆 Top 5 Products:")
print(top_products[['product_id', 'name', 'total_revenue']].head().to_string(index=False))



# %% [markdown]
# # 5. Stockout Analysis
# 
# **CRITICAL**: Where is the baseline system failing?

# %% Stockouts by product
stockout_analysis = db.query("""
    SELECT 
        i.product_id,
        p.name,
        p.category,
        COUNT(*) as total_days,
        SUM(CASE WHEN i.stockout = 1 THEN 1 ELSE 0 END) as stockout_days,
        SUM(CASE WHEN i.stockout = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as stockout_pct,
        AVG(i.units_on_hand) as avg_inventory,
        p.unit_price * p.base_demand_daily as daily_revenue_potential
    FROM inventory_snapshots i
    JOIN products p ON i.product_id = p.product_id
    GROUP BY i.product_id, p.name, p.category, p.unit_price, p.base_demand_daily
    HAVING stockout_pct > 0
    ORDER BY stockout_days DESC
    LIMIT 20
""")

fig = px.bar(
    stockout_analysis,
    x='name',
    y='stockout_days',
    color='category',
    title='Top 20 Products by Stockout Days',
    labels={'stockout_days': 'Days Out of Stock', 'name': 'Product'},
    text='stockout_pct'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(xaxis_tickangle=-45, height=600)
fig.show()

print("\n⚠️  Worst Stockout Products:")
print(stockout_analysis[['product_id', 'name', 'stockout_days', 'stockout_pct']].head(10).to_string(index=False))

# Calculate lost revenue
stockout_analysis['estimated_lost_revenue'] = (
    stockout_analysis['stockout_days'] * 
    stockout_analysis['daily_revenue_potential']
)

total_lost = stockout_analysis['estimated_lost_revenue'].sum()
print(f"\n💰 Estimated Lost Revenue from Stockouts: ${total_lost:,.2f}")



# %% Stockouts by warehouse
warehouse_stockouts = db.query("""
    SELECT 
        i.warehouse_id,
        COUNT(*) as total_snapshots,
        SUM(CASE WHEN i.stockout = 1 THEN 1 ELSE 0 END) as stockout_count,
        SUM(CASE WHEN i.stockout = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as stockout_pct
    FROM inventory_snapshots i
    GROUP BY i.warehouse_id
    ORDER BY stockout_pct DESC
""")

print("\n🏭 Stockouts by Warehouse:")
print(warehouse_stockouts.to_string(index=False))


# %% [markdown]
# # 6. Purchase Order Analysis
# 
# Understanding baseline ordering behavior

# %% Orders over time
orders_by_date = db.query("""
    SELECT 
        DATE_TRUNC('month', order_date) as month,
        COUNT(*) as num_orders,
        SUM(units_ordered) as total_units,
        SUM(total_cost) as total_cost,
        AVG(total_cost) as avg_order_value,
        SUM(CASE WHEN volume_discount_applied THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as discount_pct
    FROM purchase_orders
    GROUP BY month
    ORDER BY month
""")

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Purchase Orders Over Time', 'Volume Discount Capture Rate'),
    vertical_spacing=0.12
)

fig.add_trace(
    go.Bar(x=orders_by_date['month'], y=orders_by_date['num_orders'], name='Orders'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=orders_by_date['month'], y=orders_by_date['discount_pct'], 
               name='Discount %', mode='lines+markers'),
    row=2, col=1
)

fig.update_xaxes(title_text="Month", row=2, col=1)
fig.update_yaxes(title_text="Number of Orders", row=1, col=1)
fig.update_yaxes(title_text="Discount Capture %", row=2, col=1)
fig.update_layout(height=700, showlegend=False)

fig.show()


# %% Supplier performance comparison
supplier_performance = db.query("""
    SELECT 
        po.supplier_id,
        s.name as supplier_name,
        s.lead_time_days as expected_lead_time,
        s.reliability_score as expected_reliability,
        COUNT(*) as num_orders,
        SUM(po.total_cost) as total_spend,
        AVG(po.lead_time_actual) as avg_actual_lead_time,
        AVG(po.lead_time_expected) as avg_expected_lead_time,
        SUM(CASE WHEN po.on_time THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as actual_on_time_pct,
        SUM(CASE WHEN po.volume_discount_applied THEN 1 ELSE 0 END) * 100.0 / COUNT(*) as discount_rate,
        AVG(po.shipping_cost) as avg_shipping_cost
    FROM purchase_orders po
    JOIN suppliers s ON po.supplier_id = s.supplier_id
    GROUP BY po.supplier_id, s.name, s.lead_time_days, s.reliability_score
    ORDER BY total_spend DESC
""")

print("\n🚚 Supplier Performance:")
print(supplier_performance.to_string(index=False))

# Highlight discrepancies
print("\n⚠️  Reliability Gaps:")
for _, row in supplier_performance.iterrows():
    gap = row['actual_on_time_pct'] - (row['expected_reliability'] * 100)
    if abs(gap) > 5:
        print(f"   {row['supplier_id']}: Expected {row['expected_reliability']*100:.1f}%, "
              f"Actual {row['actual_on_time_pct']:.1f}% (gap: {gap:+.1f}pp)")
        
        
        
# %% [markdown]
# # 7. Key Findings Summary

# Calcular ingresos por categoría para el resumen final
category_revenue = db.query("""
    SELECT 
        p.category,
        SUM(s.revenue) as total_revenue
    FROM sales s
    JOIN products p ON s.product_id = p.product_id
    GROUP BY p.category
    ORDER BY total_revenue DESC
""")

print("\n" + "="*70)
print("KEY FINDINGS - DATA EXPLORATION")
print("="*70)

print(f"\n📊 REVENUE PATTERNS:")
print(f"   • YoY Growth: {growth_rate:.2f}%")
print(f"   • Seasonality: {peak_ratio:.2f}x variation (peak vs low)")
print(f"   • Top category: {category_revenue.iloc[0]['category']} "
      f"(${category_revenue.iloc[0]['total_revenue']:,.2f})")

print(f"\n⚠️  OPERATIONAL ISSUES:")
print(f"   • Stockout rate: {baseline['stockout_rate_pct']:.2f}%")
print(f"   • Lost revenue: ${baseline['estimated_lost_revenue']:,.2f}")
print(f"   • Products with stockouts: {len(stockout_analysis)}")
print(f"   • Worst product: {stockout_analysis.iloc[0]['name']} "
      f"({stockout_analysis.iloc[0]['stockout_days']:.0f} days)")

print(f"\n💰 PROCUREMENT:")
print(f"   • Total spend: ${baseline['total_procurement']:,.2f}")
print(f"   • Discount capture: {baseline['discount_capture_pct']:.1f}%")
print(f"   • Average order: ${baseline['avg_order_value']:,.2f}")
print(f"   • Total orders: {baseline['total_orders']:,}")

print("\n" + "="*70)
print("NEXT: Analyze specific failure scenarios")
print("="*70)